In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

In [6]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\khawa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
data = pd.read_csv("amazon.csv")

In [4]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [21]:
# Check for missing values
missing_values = data_clean.isnull().sum()

In [5]:
# Data preprocessing: Focus on relevant columns
data_clean = data[['review_title', 'review_content']]
data_clean['text'] = data_clean['review_title'] + " " + data_clean['review_content']
data_clean = data_clean[['text']]

C:\Users\khawa\AppData\Local\Temp\ipykernel_13344\3976343621.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['text'] = data_clean['review_title'] + " " + data_clean['review_content']


In [23]:
# Display information about missing values and the cleaned dataset preview
missing_values
data_clean.head()

,text
0,"Satisfied,Charging is really fast,Value for mo..."
1,"A Good Braided Cable for Your Type C Device,Go..."
2,"Good speed for earlier versions,Good Product,W..."
3,"Good product,Good one,Nice,Really nice product..."
4,"As good as original,Decent,Good one for second..."


In [7]:
# Initialize the Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

In [8]:
# Function to assign sentiment labels based on polarity scores
def assign_sentiment(text):
    score = sia.polarity_scores(text)
    if score['compound'] >= 0.05:
        return 'Positive'
    elif score['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [9]:
# Apply the sentiment function to the text column
data_clean['sentiment'] = data_clean['text'].apply(assign_sentiment)

In [32]:
# Save the processed data_clean to a CSV file for use in app.py
data_clean.to_csv('processed_data.csv', index=False)

In [11]:
# Display the sentiment distribution and preview the labeled data
data_clean['sentiment'].value_counts()

sentiment
Positive    1410
Negative      54
Neutral        1
Name: count, dtype: int64

In [12]:
data_clean.head()

,text,sentiment
0,"Satisfied,Charging is really fast,Value for mo...",Positive
1,"A Good Braided Cable for Your Type C Device,Go...",Positive
2,"Good speed for earlier versions,Good Product,W...",Positive
3,"Good product,Good one,Nice,Really nice product...",Positive
4,"As good as original,Decent,Good one for second...",Positive


In [14]:
positive_class = data_clean[data_clean['sentiment'] == 'Positive']
negative_class = data_clean[data_clean['sentiment'] == 'Negative']

negative_oversampled = resample(negative_class, 
                                replace=True, 
                                n_samples=len(positive_class), 
                                random_state=42)

balanced_data = pd.concat([positive_class, negative_oversampled])
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
# Display the new class distribution
balanced_distribution = balanced_data['sentiment'].value_counts()
balanced_data.head() , balanced_distribution

(                                                text sentiment
 0  Highly displayed,Very convenient for egg boili...  Positive
 1  just a watch not smart one... all features are...  Negative
 2  loud & clear,Avrage in budget range,Best for b...  Positive
 3  Nice product,Removes the lightest of lint,Very...  Positive
 4  Usable,Good,Awesome,Satisfied,Purifier,Beware ...  Negative,
 sentiment
 Positive    1410
 Negative    1410
 Name: count, dtype: int64)

In [16]:
# Step 1: Vectorization using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X = tfidf_vectorizer.fit_transform(balanced_data['text']).toarray()
y = balanced_data['sentiment']

In [17]:
# Encode the target variable
y = y.map({'Positive': 1, 'Negative': 0})

In [18]:
# Step 2: Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [19]:
# Step 3: Train a Logistic Regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [20]:
# Step 4: Evaluate the model
y_pred = model.predict(X_test)
classification_report_results = classification_report(y_test, y_pred)
confusion_matrix_results = confusion_matrix(y_test, y_pred)

classification_report_results

'              precision    recall  f1-score   support\n\n           0       0.98      1.00      0.99       282\n           1       1.00      0.98      0.99       282\n\n    accuracy                           0.99       564\n   macro avg       0.99      0.99      0.99       564\nweighted avg       0.99      0.99      0.99       564\n'

In [21]:
confusion_matrix_results

array([[282,   0],
       [  5, 277]])

In [22]:
# Perform cross-validation to check for overfitting
cv_scores = cross_val_score(model, X, y, cv=5, scoring='f1')
mean_f1 = np.mean(cv_scores)
std_f1 = np.std(cv_scores)

In [23]:
mean_f1, std_f1

(np.float64(0.9924865284210458), np.float64(0.0032958505479964133))

In [31]:
# Create requirements file
with open('requirements.txt', 'w') as f:
    f.write("pandas\n")
    f.write("numpy\n")
    f.write("scikit-learn\n")
    f.write("nltk\n")
    f.write("streamlit\n")
    f.write("matplotlib\n")
    f.write("seaborn\n")

In [33]:
# Create Streamlit app
with open('app.py', 'w') as f:
    f.write("import streamlit as st\n")
    f.write("import pandas as pd\n")
    f.write("import matplotlib.pyplot as plt\n")
    f.write("import seaborn as sns\n")
    f.write("from nltk.sentiment import SentimentIntensityAnalyzer\n")
    f.write("st.set_page_config(page_title='Sentiment Analysis', layout='wide')\n")
    f.write("st.sidebar.title('Visualization')\n")
    f.write("st.sidebar.write('The graph below shows sentiment distribution.')\n")
    f.write("# Load processed data\n")
    f.write("data_clean = pd.read_csv('processed_data.csv')\n")
    f.write("fig, ax = plt.subplots(figsize=(4, 3))\n")
    f.write("sns.countplot(x='sentiment', data=data_clean, palette='bright', ax=ax)\n")
    f.write("ax.set_title('Sentiment Distribution')\n")
    f.write("st.sidebar.pyplot(fig)\n")
    f.write("\n")
    f.write("st.title('Sentiment Analysis App')\n")
    f.write("st.write('Type your review below and the app will analyze its sentiment between positive & negative.')\n")
    f.write("review = st.text_area('Enter your review:', on_change=None, placeholder='Type here and press Enter')\n")
    f.write("if review:\n")
    f.write("    sia = SentimentIntensityAnalyzer()\n")
    f.write("    score = sia.polarity_scores(review)['compound']\n")
    f.write("    if score > 0:\n")
    f.write("        sentiment = 'Positive'\n")
    f.write("    elif score < 0:\n")
    f.write("        sentiment = 'Negative'\n")
    f.write("    else:\n")
    f.write("        sentiment = 'Neutral'\n")
    f.write("    st.write(f'The sentiment of the review is: {sentiment}')\n")
    f.write("\n")
    f.write("    # Visualize the sentiment scores\n")
    f.write("    labels = ['Positive', 'Negative', 'Neutral']\n")
    f.write("    scores = [score if sentiment == 'Positive' else 0,\n")
    f.write("              -score if sentiment == 'Negative' else 0,\n")
    f.write("              1 - abs(score) if sentiment == 'Neutral' else 0]\n")
    f.write("    fig, ax = plt.subplots(figsize=(6, 4))\n")
    f.write("    sns.barplot(x=labels, y=scores, palette='bright', ax=ax)\n")
    f.write("    ax.set_title('Sentiment Score Breakdown')\n")
    f.write("    st.pyplot(fig)\n")